In [14]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [2]:
# Получение списка ссылок

URL="https://dnd.su/bestiary/"

response = requests.get(URL)

if response.status_code == 200:

    # Парсим HTML при помощи Beautiful Soup
    soup = BeautifulSoup(response.text, 'html.parser')

bestiary_href = []

for data in soup.findAll('a', class_='list-item-wrapper'):
    bestiary_href.append("https://dnd.su"+data['href'])


In [3]:
# Функция парсинга HTML страницы бестиария

def bestiary_parser(card, count, url):

    # Пустой фрейм
    df = pd.DataFrame([],columns=['id','name', 'link', 'immunity_condition', 
                                  'immunity_damage','def','hits','habitat','exp',
                                  'speed','resist','saving','debility', 'languages',
                                  'feels', 'skill_bonus', 'source','strength', 'dexterity',
                                  'constitution', 'intelligence', 'wisdom', 'charisma', 'brief_inf'])
    
    try: # try вне цикла т.к. ошибка вознивает в findAll()
        #Цикл по всем "li" с нахождением нужного текста
        for card_params in card.findAll('li', class_=''):
    
            if 'Класс Доспеха' in card_params.find('strong').text:
                number = ''.join(c if c.isdigit() else ' ' for c in card_params.text).split()
                df.at[count, 'def'] = int(number[0])
                
            if 'Бонус мастерства' in card_params.find('strong').text:
                df.at[count, 'skill_bonus'] = card_params.text.replace('Бонус мастерства ', '')
    
            if 'Чувства' in card_params.find('strong').text:
                df.at[count, 'feels'] = card_params.text.replace('Чувства ', '')
    
            if 'Языки' in card_params.find('strong').text:
                df.at[count, 'languages'] = card_params.text.replace('Языки ', '')
    
            if 'Иммунитет к состоянию' in card_params.find('strong').text:
                df.at[count, 'immunity_condition']  = card_params.text.replace('Иммунитет к состоянию ', '')
    
            if 'Иммунитет к урону' in card_params.find('strong').text:
                df.at[count, 'immunity_damage']  = card_params.text.replace('Иммунитет к урону ', '')
    
            if 'Хиты' in card_params.find('strong').text:
                df.at[count, 'hits'] = card_params.text.replace('Хиты ', '')
    
            if 'Местность обитания' in card_params.find('strong').text:
                df.at[count, 'habitat'] = card_params.text.replace('Местность обитания ', '')
    
            if 'Опасность' in card_params.find('strong').text:
                df.at[count, 'exp'] = card_params.text.replace('Опасность ', '')
    
            if 'Скорость' in card_params.find('strong').text:
                df.at[count, 'speed'] = card_params.text.replace('Скорость ', '')
    
            if 'Сопротивление урону' in card_params.find('strong').text:
                df.at[count, 'resist'] = card_params.text.replace('Сопротивление урону ', '')
    
            if 'Спасброски' in card_params.find('strong').text:
                df.at[count, 'saving'] = card_params.text.replace('Спасброски ', '')
    
            if 'Уязвимость к урону' in card_params.find('strong').text:
                df.at[count, 'debility'] = card_params.text.replace('Уязвимость к урону ', '')
    
            if 'Источник:' in card_params.find('strong').text:
                df.at[count, 'source'] = card_params.text.replace('Источник: ', '')
                break
    
            if 'Источники:' in card_params.find('strong').text:
                df.at[count, 'source'] = card_params.text.replace('Источники: ', '')
                break
                    
    except Exception as e: 
        print(f'id={count+1} Ошибка {e}')

    df.at[count, 'id'] = count+1
    df.at[count, 'name'] = card.find('h2', class_='card-title').text
    df.at[count, 'link'] = url
    df.at[count, 'brief_inf'] = card.find('li', class_='size-type-alignment').text
        
    #Статы       
    for stats in card.findAll('div', class_='stat'):
        number = ''.join(c if c.isdigit() else ' ' for c in stats.text).split()
        number[0]=int(number[0])
        #Отрицательные числа
        if number[0]<=9:
           number[1]=-int(number[1])
        else: number[1]=int(number[1])
        if 'Сил' in stats.text:
            df.at[count, 'strength'] = number
        if 'Лов' in stats.text:
            df.at[count, 'dexterity'] = number
        if 'Тел' in stats.text:
            df.at[count, 'constitution'] = number
        if 'Инт' in stats.text:
            df.at[count, 'intelligence'] = number
        if 'Мдр' in stats.text:
            df.at[count, 'wisdom'] = number
        if 'Хар' in stats.text:
            df.at[count, 'charisma'] = number

    return df


In [7]:
# Функция возвращающая HTML страницы бестиария

def get_bestiary_html(url):
    
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.find('div', class_='cards-wrapper')
    else: return 0    


In [5]:
# Создание пустого фрейма для бестиария

bestiary_df = pd.DataFrame([],columns=['id','name', 'link', 'immunity_condition', 
                                  'immunity_damage','def','hits','habitat','exp',
                                  'speed','resist','saving','debility', 'languages',
                                  'feels', 'skill_bonus', 'source','strength', 'dexterity',
                                  'constitution', 'intelligence', 'wisdom', 'charisma', 'brief_inf'])


In [8]:
# Цикл для заполнение фрейма бестиария

count=0

for url in bestiary_href:
    try:
        card= get_bestiary_html(url)
        bestiary_df = pd.concat([bestiary_df, bestiary_parser(card, count, url)], ignore_index=True)
    except:
        time.sleep(10)
        card= get_bestiary_html(url)
        bestiary_df = pd.concat([bestiary_df, bestiary_parser(card, count, url)], ignore_index=True)

    count+=1

id=1150 Ошибка 'NoneType' object has no attribute 'findAll'


In [28]:
# сохранение данных
bestiary_df.to_csv('Data/bestiary.csv', index=False)

In [29]:
# загрузка данных для проверки
bestiary_df = pd.read_csv('Data/bestiary.csv')

In [30]:
# Проверка данных

if len(bestiary_href) == len(bestiary_df): # Проверка длины фрейма и длины списка ссылок
    print('Фрейм соответствует по длине\n')

duplicates_series = bestiary_df.duplicated().value_counts()

if duplicates_series[duplicates_series.index == False].iloc[0] == len(bestiary_df): # Проверка на дублирующие строки
    print('Дубли строк отсутсвуют\n')
else: print('Дубли строк присутствуют\n')

none_series = bestiary_df['name'].isna().value_counts()

if none_series[none_series.index == False].iloc[0] == len(bestiary_df): # Проверка на нулевые имена
    print('Нулевые имена отсутствуют\n')
else: print('Нулевые имена присутствуют\n')


Фрейм соответствует по длине

Дубли строк отсутсвуют

Нулевые имена отсутствуют

